In [2]:
# hocemo da iskucamo onaj primer sa automobilom (za pocetak na najgluplji moguci nacin da radi samo za
# taj primer i eventualno slicne njemu)
# trebaju nam ulazne promenljive, izlazne prom,enljive i pravila, pravimo po klasu za njih

class FuzzyInput():
    # prosledjujemo ime promenljice i tacke u kojima se menja da li funkcija raste ili opada
    # x0 je target (ono sto racunamo za onaj konkretan automobil (?))
    def __init__(self, name, xs, ys, x0):
        self.name = name
        self.points = list(zip(xs, ys))
        # ono sto smo zvali 'mi', funkcija pripadnosti
        self.mu = self.calcMu(x0)
    
    def calcMu(self, x0):
        if x0 < self.points[0][0]:       # ako je levo od najlevlje tacke
            return self.points[0][1]
        elif x0 > self.points[-1][0]:  # desno od najdesnjeg
            return self.points[-1][1]
        for i in range(len(self.points) - 1):  # slucajevi gde je izmedju 2 tacke
            x1 = self.points[i][0]
            x2 = self.points[i+1][0]
            if x0 >= x1 and x0 < x2:
                y1 = self.points[i][1]
                y2 = self.points[i+1][1]
                if y1 == y2:
                    return y1
                elif y1 < y2:   # slicnost trougla
                    return (x0 - x1) / (x2 - x1)
                else:
                    return (x2 - x0) / (x2 - x1)

In [4]:
x = FuzzyInput('ime', [3,10], [1,0], 9)
print(x.mu)

0.14285714285714285


In [7]:
class FuzzyOutput:
    def __init__(self, name, xs, ys):
        self.name = name
        self.xs = xs
        self.ys = ys
        self.mu = 0    # i ovde postoji mi, ali ovde ne znamo sta je (?) a imacemo posle neku klasu za pravila (?)
        self.c = 0  # c je u nasem primeru bila neka karakteristicna tacka (?), trazimo prosek x-eva za tacke gde je y=1
        n = 0
        for x, y in zip(self.xs, self.ys):
            if y == 1:
                self.c += x
                n += 1
        self.c /= n     

In [9]:
x = FuzzyOutput('iz', [7,15], [1,0])
x.c

7.0

In [14]:
# koristimo samo pravila iz primera koji smo radili, npr kodiramo pravila samo oblika
# input1 operator input2 => output, gde ce operator da nam bude konjukcija ili disjunkcija (koju cemo da
#napravimo kao enum)

# ovako se radi sa enumima u pajtonu, mora prvo import pa onda pravimo klasu koja nasledjuje Enum
from enum import Enum

class LogicOp(Enum):
    AND = 0
    OR = 1

class Rule:   # input1 i input2 ce biti onog FuzzyInput tipa
    def __init__(self, input1, input2, output, operator):
        self.input1 = input1
        self.input2 = input2
        self.operator = operator
        self.output = output
        if operator == LogicOp.AND:
            self.output.mu = max(self.output.mu, min(self.input1.mu, self.input2.mu))
        else:
            self.output.mu = max(self.output.mu, max(self.input1.mu, self.input2.mu))

In [19]:
potrosnja = []
potrosnja.append(FuzzyInput('mala', [3,10], [1,0], 9)) 
potrosnja.append(FuzzyInput('srednja', [7,10,12,15], [0,1,1,0], 9)) 
potrosnja.append(FuzzyInput('velika', [12,15], [0,1], 9))  

pouzdanost = []
pouzdanost.append(FuzzyInput('visoka', [5,10], [1,0], 8))
pouzdanost.append(FuzzyInput('niska', [8,15], [0,1], 8))

vrednost = []
vrednost.append(FuzzyOutput('mala', [7,15], [1,0]))
vrednost.append(FuzzyOutput('srednja', [7,15,25,40], [0,1,1,0]))
vrednost.append(FuzzyOutput('velika', [25,40], [0,1]))

rules = []
rules.append(Rule(potrosnja[0], pouzdanost[0], vrednost[2], LogicOp.AND))
rules.append(Rule(potrosnja[0], pouzdanost[1], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[1], pouzdanost[0], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[1], pouzdanost[1], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[2], pouzdanost[0], vrednost[1], LogicOp.AND))
rules.append(Rule(potrosnja[2], pouzdanost[1], vrednost[0], LogicOp.AND))

In [21]:
# for v in vrednost:
#    print(v.mu)

# defazifikacija
iznad = 0  # iznad razloamcke crte
ispod = 0  # ispid razloamcke crte
for v in vrednost:
    iznad += v.c * v.mu
    ispod += v.mu

print(iznad / ispod)
    

25.26315789473684
